# <center> Transport Layer (TLS) Deep Dive
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily<br>
### <center> Content Author: Jerry Duggan

## Learning Objectives
By the end of this module, you will:
* Understand how to create and verify a self-signed certificate;
* Understand how to initiate a TLS v1.2 session with an HTTP server;
* Understand how to access components of a TLS certificate with a client application, and how to verify the certificate;
* Understand how to use Wireshark to examine the steps for establishing a TLS session.

### What is Transport Layer Security (TLS)
Transport Layer Security (TLS) is a cryptographic protocol that provides secure communication over the internet by encrypting data between two communicating applications. It is commonly used to secure web traffic, email, instant messaging, and other types of communication.

TLS operates at the transport layer of the OSI model, which is responsible for providing reliable data transfer between applications. TLS uses a combination of symmetric and asymmetric cryptography to provide confidentiality, integrity, and authentication of data exchanged between the communicating applications.

TLS has become the de facto standard for securing web traffic and is used by most modern web browsers and servers. It is an essential part of online security and is used to protect sensitive information such as passwords, credit card numbers, and personal data. TLS is constantly evolving to improve security and address new threats.

The Transport Layer Security (TLS) protocol is defined in a series of RFC documents published by the Internet Engineering Task Force (IETF). The current version of TLS, TLS 1.3, is defined in RFC 8446, which was published in August 2018. Previous versions of TLS were defined in RFC 5246 (TLS 1.2), RFC 4346 (TLS 1.1), RFC 2246 (TLS 1.0), and RFC 6101 (SSL 3.0).

Here is a high level sequence diagram of the communications for secure communication s session. The diagram is made with the Mermaid website: www.mermaid.live.
```
sequenceDiagram
    participant Client
    participant Server
    Client->>Server: ClientHello
    Server->>Client: ServerHello, Certificate, CertificateRequest, ServerHelloDone
    Client->>Server: Certificate, CertificateVerify, ClientKeyExchange, ChangeCipherSpec, Finished
    Server->>Client: ChangeCipherSpec, Finished
    Note over Client,Server: Encrypted Application Data
```
<center> <img src="TLS_1_2_Sequence_Diagram.svg" width="75%" /> </center>
In this diagram, the client initiates a TLS 1.3 handshake by sending a ClientHello message to the server. The server responds with a ServerHello message, which includes its certificate and a request for the client's certificate. After exchanging some additional messages, both the client and server switch to using the new cryptographic keys established during the handshake and exchange Finished messages to verify that the handshake was successful.

Once the handshake is complete, the client and server can exchange encrypted application data. This is represented in the diagram by the note "Encrypted Application Data" above both the client and server participants.

## Attacking Encrypted Communications
Breaking a cipher is really hard, so attacks usually focus on recovery of key material to decrypt encrypted traffic.

Here's a scenario: The attacker intercepts the ClientHello message from the client and responds with a fake ServerHello message pretending to be the server. The attacker then intercepts the ClientKeyExchange message from the client and responds with a fake ClientKeyExchange message pretending to be the client. The attacker continues to intercept and respond with fake messages throughout the handshake process, effectively being in the middle of the communication between the client and server.

Here's the mermaid code and drawing:

```
sequenceDiagram
    participant Client
    participant Attacker
    participant Server

    Client->>+Server: ClientHello
    Note over Attacker: Attacker intercepts the message
    Attacker->>+Client: ServerHello (fake)
    Note over Attacker: Attacker pretends to be the server
    Client->>+Attacker: ClientKeyExchange
    Attacker->>+Server: ClientKeyExchange (fake)
    Note over Attacker: Attacker pretends to be the client
    Server->>+Attacker: ServerHelloDone
    Attacker->>+Client: ServerHelloDone (fake)
    Client->>+Attacker: ChangeCipherSpec
    Attacker->>+Server: ChangeCipherSpec (fake)
    Client->>+Attacker: Finished
    Attacker->>+Server: Finished (fake)
    Server-->>-Attacker: Encrypted handshake messages
    Attacker-->>-Client: Encrypted handshake messages
```
<center> <img src="AttackerSequenceDiagram.svg" alt="Attacker Sequence Diagram" width="75%" /> </center>

## Establishing Trust
To trust a certificate from a server, the client needs to verify that the certificate is issued by a trusted certificate authority (CA) and that the certificate is still valid and has not been revoked.

When a client receives a certificate from a server during a TLS handshake, it must perform the following steps to verify the certificate:

1. Validate the digital signature of the certificate to ensure the contents have not been altered. 
2. Check the certificate's expiration date to ensure that it has not expired.
3. Verify that the hostname of the server matches the hostname listed in the certificate.
4. Verify that the certificate is issued by a trusted CA. This can be done by checking that the issuer of the certificate is listed in the client's list of trusted root CAs, or by checking the signature chain of the certificate up to a trusted root CA.
5. Check if the certificate has been revoked by the issuing CA. This can be done by checking the certificate revocation list (CRL) published by the CA or by using the Online Certificate Status Protocol (OCSP).

If any of these steps fail, the client should not trust the certificate and should terminate the TLS handshake.


Fortunately, most modern browsers do this behind the scenes. Let's look at doing some of these in this notebook.

Mermaid Code:
```
sequenceDiagram
    participant Certificate Authority
    participant Server
    participant Client
    
    Note over Certificate Authority,Trust Store:Start One-time Operation, Root Certificates last a long time
    Certificate Authority->>Certificate Authority: Generate Self-Signed Certificate
    Certificate Authority->>Trust Store: Submit Root Signing Certificate
    Note over Certificate Authority,Trust Store:End One-time Operation, Root Certificates last a long time
    Note over Certificate Authority,Server: Start Periodic Operation (e.g. annual)
    Server->>Server: Generate fresh asymmetric key pair
    Server->>Certificate Authority: Certificate Signing Request (CSR)
    Certificate Authority->>Certificate Authority: Verify the Server
    Certificate Authority->>Server: Signed Certificate
    Note over Certificate Authority,Server: End Periodic Operation (e.g. annual)
    Note over Client,Trust Store: Start infrequent updates
    Client->>Trust Store: Request CA Certificates
    Trust Store->>Client: Trusted CA Certificates
    Note over Client,Trust Store: End infrequent updates
    Note over Client,Server: Every session
    Client->>Server: ClientHello
    Server->>Client: ServerHello
    Server->>Client: Certificate
    Client->>Client: Verify server certificate digital signature
    Client->>Client: Verify certificate hostname, date
    Client->>Client: Verify certificate chain
    Client->>Certificate Authority: Check CA Root Certificate Revocation Status
    Certificate Authority->>Client: Update Certificate Revocation List
    Client->>Server: Certificate
    Client->>Server: ClientKeyExchange
    Note over Server,Client: Encrypted Session
```

<center> <img src="FullSequenceDiagram.svg" alt="Full Public Key Infrastructure" width="40%" /> </center>

## Generating and Using a Certificate
This tutorial follows along with the approach found at https://cryptography.io/en/latest/x509/.

We'll start out by creating a certificate and putting together a certificate signing request.


### Load dependencies
This was written with crytography.io at version 40.

In [1]:
%pip install --upgrade --user cryptography

Requirement already up-to-date: cryptography in c:\users\jdaily\appdata\roaming\python\python38\site-packages (40.0.1)
Note: you may need to restart the kernel to use updated packages.


In [76]:
#Load the modules needed for this notebook
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography import x509
from datetime import datetime, timedelta
import requests
import certifi
import socket
import ssl

### Generate private key & calculate it's associated public key

In [3]:
# Generate a unique private key
privateKey = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
publicKey = privateKey.public_key()
print(publicKey)

### Create an x509 object with our pertinent information

In [4]:
#Build an x509 Name object for us
issuer = x509.Name([
        x509.NameAttribute(x509.NameOID.COUNTRY_NAME,           u"US"),
        x509.NameAttribute(x509.NameOID.STATE_OR_PROVINCE_NAME, u"Colorado"),
        x509.NameAttribute(x509.NameOID.LOCALITY_NAME,          u"Fort Collins"),
        x509.NameAttribute(x509.NameOID.ORGANIZATION_NAME,      u"Colorado State University"),
        x509.NameAttribute(x509.NameOID.COMMON_NAME,            u"fakesite.edu")
    ])
issuer

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

### For a self-signed certificate, the issuer is also the subject

In [5]:
subject = issuer

### Create the certificate using a Certificate builder

In [6]:
#Assemble the certificate parts
rawCert = (
        x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(issuer)
            .public_key(publicKey)
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.utcnow())
            .not_valid_after(datetime.utcnow() + timedelta(days=10))
            .add_extension(x509.SubjectAlternativeName([x509.DNSName(u"localhost")]), critical=False)
            )
rawCert

### Sign it with our private key to make it a real certificate

In [7]:
#Sign the certificate with our own private key.
signedCert = rawCert.sign(privateKey, hashes.SHA256())
signedCert

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>, ...)>

In [8]:
#Export the certificate to save or send
signedCert_PEM = signedCert.public_bytes(serialization.Encoding.PEM)
print(signedCert_PEM)

b'-----BEGIN CERTIFICATE-----\nMIIDijCCAnKgAwIBAgIUURTYb7q1cCVIFnKSRty8a2o6ufkwDQYJKoZIhvcNAQEL\nBQAwcjELMAkGA1UEBhMCVVMxETAPBgNVBAgMCENvbG9yYWRvMRUwEwYDVQQHDAxG\nb3J0IENvbGxpbnMxIjAgBgNVBAoMGUNvbG9yYWRvIFN0YXRlIFVuaXZlcnNpdHkx\nFTATBgNVBAMMDGZha2VzaXRlLmVkdTAeFw0yMzA0MDgyMDAyNTBaFw0yMzA0MTgy\nMDAyNTBaMHIxCzAJBgNVBAYTAlVTMREwDwYDVQQIDAhDb2xvcmFkbzEVMBMGA1UE\nBwwMRm9ydCBDb2xsaW5zMSIwIAYDVQQKDBlDb2xvcmFkbyBTdGF0ZSBVbml2ZXJz\naXR5MRUwEwYDVQQDDAxmYWtlc2l0ZS5lZHUwggEiMA0GCSqGSIb3DQEBAQUAA4IB\nDwAwggEKAoIBAQCfDnMUaFvl3fL8/0LqdZLBklNzAopa/+bAFj+LFHzanq2iIU3U\n5ooDYDSBGd2dUNCGH72eO5OKNXlCQg9xzFH7gBkuu4yzLXMGUq1TVnvh5TsDWT4u\nJ56TgTM7I2ExVP8dQ0ETunLvCWuSQW/wU3Uko0ykGO27JD05olU5vLt5UcPEXyXT\nldJNQ2+tSDeNwfy6jnOnZMXy9RES6rqJvAp25RCOewpFvuirl61ooaqsfoyx+4xP\n7a/2Wbk0ABCCtfwlg3Ihk8tcAyOJi26Y2aoL2S2Skt4xmiahfYWPiwTU7fCDJYS8\nx2Bz990zQ2uxOxUWX6rNQTF1FXx1h2qZ3f7bAgMBAAGjGDAWMBQGA1UdEQQNMAuC\nCWxvY2FsaG9zdDANBgkqhkiG9w0BAQsFAAOCAQEAQ2YUkZULhRXZUkDUp/X6nSa0\n14cCRboI85tvmUmjSK0NP/ggPozHMShHsHBUF6Jw8znd1

### Pretend the public key & certificate were transmitted...

In [9]:
# Instead of sending it, we'll just save it to disk
with open('cert.pem','wb') as f:
    f.write(signedCert_PEM)

### Pretend the client receives the certificate...

In [10]:
# Instead of receiveing it, we;ll just open it from disk
with open('cert.pem','rb') as f:
    received_pem = f.read()
print(received_pem)

b'-----BEGIN CERTIFICATE-----\nMIIDijCCAnKgAwIBAgIUURTYb7q1cCVIFnKSRty8a2o6ufkwDQYJKoZIhvcNAQEL\nBQAwcjELMAkGA1UEBhMCVVMxETAPBgNVBAgMCENvbG9yYWRvMRUwEwYDVQQHDAxG\nb3J0IENvbGxpbnMxIjAgBgNVBAoMGUNvbG9yYWRvIFN0YXRlIFVuaXZlcnNpdHkx\nFTATBgNVBAMMDGZha2VzaXRlLmVkdTAeFw0yMzA0MDgyMDAyNTBaFw0yMzA0MTgy\nMDAyNTBaMHIxCzAJBgNVBAYTAlVTMREwDwYDVQQIDAhDb2xvcmFkbzEVMBMGA1UE\nBwwMRm9ydCBDb2xsaW5zMSIwIAYDVQQKDBlDb2xvcmFkbyBTdGF0ZSBVbml2ZXJz\naXR5MRUwEwYDVQQDDAxmYWtlc2l0ZS5lZHUwggEiMA0GCSqGSIb3DQEBAQUAA4IB\nDwAwggEKAoIBAQCfDnMUaFvl3fL8/0LqdZLBklNzAopa/+bAFj+LFHzanq2iIU3U\n5ooDYDSBGd2dUNCGH72eO5OKNXlCQg9xzFH7gBkuu4yzLXMGUq1TVnvh5TsDWT4u\nJ56TgTM7I2ExVP8dQ0ETunLvCWuSQW/wU3Uko0ykGO27JD05olU5vLt5UcPEXyXT\nldJNQ2+tSDeNwfy6jnOnZMXy9RES6rqJvAp25RCOewpFvuirl61ooaqsfoyx+4xP\n7a/2Wbk0ABCCtfwlg3Ihk8tcAyOJi26Y2aoL2S2Skt4xmiahfYWPiwTU7fCDJYS8\nx2Bz990zQ2uxOxUWX6rNQTF1FXx1h2qZ3f7bAgMBAAGjGDAWMBQGA1UdEQQNMAuC\nCWxvY2FsaG9zdDANBgkqhkiG9w0BAQsFAAOCAQEAQ2YUkZULhRXZUkDUp/X6nSa0\n14cCRboI85tvmUmjSK0NP/ggPozHMShHsHBUF6Jw8znd1

In [11]:
# Let's load the PEM file into a certificate object
received_cert = x509.load_pem_x509_certificate(received_pem)
received_cert

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>, ...)>

In [12]:
#This should match what we just loaded.
received_cert.public_bytes(serialization.Encoding.PEM) == signedCert_PEM

True

### X509 Data and objects
What things are available in the certificate?

In [13]:
#Version for the certificate
received_cert.version

<Version.v3: 2>

In [14]:
#The certificate fingerprint, which is easier to look up
fingerprint = received_cert.fingerprint(hashes.SHA256())
"".join(["{:02X}".format(b) for b in fingerprint])

'4B4EE480F4CC5D0A8FCF80240512183CC9426AB53A76422ED6265EDBA593DD64'

In [15]:
#The random certificate serial number
received_cert.serial_number

462893121613984089894177319962981681438662507001

In [16]:
# Extract the public key from the certificate. This is critical to verify the signature
cert_pub_key = received_cert.public_key()
cert_pub_key

In [17]:
# A datetime object to show the birth of the certificate
received_cert.not_valid_before

datetime.datetime(2023, 4, 8, 20, 2, 50)

In [18]:
# A datetime object to show the expiration of the certificate
received_cert.not_valid_after

datetime.datetime(2023, 4, 18, 20, 2, 50)

In [19]:
# A Name object identifying the issuer of the certificate
received_cert.issuer

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

In [20]:
# A Name object identifying the subject of the certificate.
received_cert.subject

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

In [21]:
# The algorithm object created by the library for use in hashing the certificate contents
received_cert.signature_hash_algorithm

In [22]:
# The object identifier for the signature algorithm
received_cert.signature_algorithm_oid

<ObjectIdentifier(oid=1.2.840.113549.1.1.11, name=sha256WithRSAEncryption)>

In [23]:
# The bytes for the actual signature
received_cert.signature

b'Cf\x14\x91\x95\x0b\x85\x15\xd9R@\xd4\xa7\xf5\xfa\x9d&\xb4\xd7\x87\x02E\xba\x08\xf3\x9bo\x99I\xa3H\xad\r?\xf8 >\x8c\xc71(G\xb0pT\x17\xa2p\xf39\xdd\xd5\xce\x1d\xf4\x8aH_\xe5D \x107\x00\xdf\x07\xcb\xbd\x11\n\x96H\x85J\xc9\xc4Nk\x85\\\xf3c\xcf\xd8^\xe8\xefx\xbc\x18\x90\x0e\xa2{\x1a\rJ\xe6\xe14\xc2\xf1\x1cs\x8eVZC\x94\xda`\xd4\xbd\x86\xff\x03>v\xd3\xe8u\x00 \x81]y\xa6\xb6\xaeNJ\xb17\x874\x82M\x8d\x04\xc2\xdcj@\xe4\xa6\x8e\x1f\x1a\xa7\x98\xeb\x8dl\r\xa2\x10(E\x1fe\xbeC\xc0_\x93\xb6tN\xf7@\xf8\xf6[\xad\xafy\xd9\xc6\x12\x19u0\xf8\xd2\xa5\xc4\xd0\xbeO\x7fP)\x89\xda\x9aer#Y\x01j\xefb\x12h\xee)\xab i\x8e\x9c\xad\xbaPT\xce\xf8\x14c\x1b]\xdb\x1b\x94X\x84\x9b\x15F\xdeD\xd8\x19\x0eL\x95\x07I\xfd\xa5H\xe57\xa9\xfb\xff\xaa{\x8d\x88\x04\x9c\xb3\xba\x03'

In [24]:
#Enumerate the x509 extensions
for ext in received_cert.extensions:
    print(ext)

<Extension(oid=<ObjectIdentifier(oid=2.5.29.17, name=subjectAltName)>, critical=False, value=<SubjectAlternativeName(<GeneralNames([<DNSName(value='localhost')>])>)>)>


There is a big list of structured constants, known as Object Identifiers, that are enumerated in the documentation.

https://cryptography.io/en/latest/x509/reference/#object-identifiers

Let's get the value of the DNSName.

In [25]:
DNSNameOID = x509.ExtensionOID.SUBJECT_ALTERNATIVE_NAME
ext = received_cert.extensions.get_extension_for_oid(DNSNameOID)
ext.value.get_values_for_type(x509.DNSName)

['localhost']

In [26]:
#The DER encoded bytes payload (as defined by RFC 5280) is hashed and then
#signed by the private key of the certificate’s issuer.
received_cert.tbs_certificate_bytes

b'0\x82\x02r\xa0\x03\x02\x01\x02\x02\x14Q\x14\xd8o\xba\xb5p%H\x16r\x92F\xdc\xbckj:\xb9\xf90\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000r1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x110\x0f\x06\x03U\x04\x08\x0c\x08Colorado1\x150\x13\x06\x03U\x04\x07\x0c\x0cFort Collins1"0 \x06\x03U\x04\n\x0c\x19Colorado State University1\x150\x13\x06\x03U\x04\x03\x0c\x0cfakesite.edu0\x1e\x17\r230408200250Z\x17\r230418200250Z0r1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x110\x0f\x06\x03U\x04\x08\x0c\x08Colorado1\x150\x13\x06\x03U\x04\x07\x0c\x0cFort Collins1"0 \x06\x03U\x04\n\x0c\x19Colorado State University1\x150\x13\x06\x03U\x04\x03\x0c\x0cfakesite.edu0\x82\x01"0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x82\x01\x0f\x000\x82\x01\n\x02\x82\x01\x01\x00\x9f\x0es\x14h[\xe5\xdd\xf2\xfc\xffB\xeau\x92\xc1\x92Ss\x02\x8aZ\xff\xe6\xc0\x16?\x8b\x14|\xda\x9e\xad\xa2!M\xd4\xe6\x8a\x03`4\x81\x19\xdd\x9dP\xd0\x86\x1f\xbd\x9e;\x93\x8a5yBB\x0fq\xccQ\xfb\x80\x19.\xbb\x8c\xb3-s\x06R\xadSV{\xe1\xe5;\x03Y>.\'\x9e\x93\x813;#a1T\xf

### And now the "client" verifies  that the certificate has not been altered.

In [27]:
#The verify function will throw an execption if the signature is not valid
cert_pub_key.verify(
        received_cert.signature,
        received_cert.tbs_certificate_bytes,
        padding.PKCS1v15(),
        received_cert.signature_hash_algorithm
    )
print("Valid signature")

Valid signature


In [28]:
#We get an exception if the certificate or signature are altered.
from cryptography.exceptions import InvalidSignature
try:
    cert_pub_key.verify(
        received_cert.signature,
        # change just the first byte to get manipulated data
        b'1' + received_cert.tbs_certificate_bytes[1:],
        padding.PKCS1v15(),
        received_cert.signature_hash_algorithm
    )
    print("Valid signature")
except InvalidSignature as e:
    print(repr(e))

InvalidSignature()


In [29]:
#Make sure the time has not expired on the certificate
datetime.utcnow() < received_cert.not_valid_after

True

In [30]:
#Make sure there the clock wasn't manipulated to show an old certificate
datetime.utcnow() > received_cert.not_valid_before

True

In [31]:
# Check for a valid time slot:
if (datetime.utcnow() < received_cert.not_valid_after and
    datetime.utcnow() > received_cert.not_valid_before):
    print("The certificate has not expired")
else:
    print("The certificate is outside the allowed times.")

The certificate has not expired


### Signing Signatures with Elliptic Curve Cryptography
Signatures that use ECC are smaller, so the certificates are also smaller.

As an exercise, repeat the signature generation, but use the ed25519 elliptic curve instead of RSA.

In [32]:
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
privateECCkey = Ed25519PrivateKey.generate()
ECCsignedCert = rawCert.sign(privateECCkey,None)
print(ECCsignedCert)
ECCsignedCert.signature

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>, ...)>


b'\xc3\xf4v\xad\xd4\xaaw f\xa5\x1d\xc7\x063\xba\x14\x8b3\xefE\x8b\x07`~VO\xedZ\xbbC\x94\x16\x8dxM\x8b\xc81\xe9[\xef\x13\xa2\x9f\n\xb6\x10\x1a.g\x83\x9e\x12)\x81\xac\xe4d\xb2\xf0\xd5<\xd6\x06'

### Issues
We have shown there is a valid self-signed certificate. An attacker can generate their own valid certificate. How do we know the certificate from a server is the correct certificate? We need to check it against a certificate chain.

## Certificate Chains
Let's load a series of root certificates at the top of a certificate chain. These are inherently trusted certificates. However, this assumption may not be always valid if someone is able to place a root certificate in a root trust store.
Or, the root certificate may be compromised. For example, see this article:

https://nakedsecurity.sophos.com/2011/03/24/fraudulent-certificates-issued-by-comodo-is-it-time-to-rethink-who-we-trust/




In [33]:
%pip install certifi

Note: you may need to restart the kernel to use updated packages.


Download and install trusted root certificates.

https://github.com/certifi/python-certifi

In [77]:
certifi.where()

'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'

In [35]:
# Read teh root_CAs into memory
with open(certifi.where(),'rb') as f:
    root_CAs = f.read()
root_CAs [:1000]

b'\n# Issuer: CN=GlobalSign Root CA O=GlobalSign nv-sa OU=Root CA\n# Subject: CN=GlobalSign Root CA O=GlobalSign nv-sa OU=Root CA\n# Label: "GlobalSign Root CA"\n# Serial: 4835703278459707669005204\n# MD5 Fingerprint: 3e:45:52:15:09:51:92:e1:b7:5d:37:9f:b1:87:29:8a\n# SHA1 Fingerprint: b1:bc:96:8b:d4:f4:9d:62:2a:a8:9a:81:f2:15:01:52:a4:1d:82:9c\n# SHA256 Fingerprint: eb:d4:10:40:e4:bb:3e:c7:42:c9:e3:81:d3:1e:f2:a4:1a:48:b6:68:5c:96:e7:ce:f3:c1:df:6c:d4:33:1c:99\n-----BEGIN CERTIFICATE-----\nMIIDdTCCAl2gAwIBAgILBAAAAAABFUtaw5QwDQYJKoZIhvcNAQEFBQAwVzELMAkG\nA1UEBhMCQkUxGTAXBgNVBAoTEEdsb2JhbFNpZ24gbnYtc2ExEDAOBgNVBAsTB1Jv\nb3QgQ0ExGzAZBgNVBAMTEkdsb2JhbFNpZ24gUm9vdCBDQTAeFw05ODA5MDExMjAw\nMDBaFw0yODAxMjgxMjAwMDBaMFcxCzAJBgNVBAYTAkJFMRkwFwYDVQQKExBHbG9i\nYWxTaWduIG52LXNhMRAwDgYDVQQLEwdSb290IENBMRswGQYDVQQDExJHbG9iYWxT\naWduIFJvb3QgQ0EwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQDaDuaZ\njc6j40+Kfvvxi4Mla+pIH/EqsLmVEQS98GPR4mdmzxzdzxtIK+6NiY6arymAZavp\nxy0Sy6scTHAHoT0KMM0VjU/43dSMUBUc71DuxC73/O

In [36]:
#There are a bunch of certificates in this store.
len(root_CAs)

282394

In [37]:
#Load all the certificate material into Python objects
root_certs = x509.load_pem_x509_certificates(root_CAs)
root_certs

C:\Users\jdaily\AppData\Roaming\Python\Python38\site-packages\cryptography\x509\base.py:576: CryptographyDeprecationWarning: Parsed a negative serial number, which is disallowed by RFC 5280.
  return rust_x509.load_pem_x509_certificates(data)


[<Certificate(subject=<Name(C=BE,O=GlobalSign nv-sa,OU=Root CA,CN=GlobalSign Root CA)>, ...)>,
 <Certificate(subject=<Name(OU=GlobalSign Root CA - R2,O=GlobalSign,CN=GlobalSign)>, ...)>,
 <Certificate(subject=<Name(O=Entrust.net,OU=www.entrust.net/CPS_2048 incorp. by ref. (limits liab.),OU=(c) 1999 Entrust.net Limited,CN=Entrust.net Certification Authority (2048))>, ...)>,
 <Certificate(subject=<Name(C=IE,O=Baltimore,OU=CyberTrust,CN=Baltimore CyberTrust Root)>, ...)>,
 <Certificate(subject=<Name(C=US,O=Entrust\, Inc.,OU=www.entrust.net/CPS is incorporated by reference,OU=(c) 2006 Entrust\, Inc.,CN=Entrust Root Certification Authority)>, ...)>,
 <Certificate(subject=<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Global CA)>, ...)>,
 <Certificate(subject=<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Universal CA)>, ...)>,
 <Certificate(subject=<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Universal CA 2)>, ...)>,
 <Certificate(subject=<Name(C=GB,ST=Greater Manchester,L=Salford,O=Comodo CA Limited,CN=AAA Cert

In [60]:
#Let's see the organizations for the of the root certificates. 
commonNameOID = x509.oid.NameOID.COMMON_NAME
for rootCA in root_certs:
    try:
        print(rootCA.issuer.get_attributes_for_oid(commonNameOID)[0].value)
    except IndexError:
        # no organization name
        print(rootCA.issuer)
    

GlobalSign Root CA
GlobalSign
Entrust.net Certification Authority (2048)
Baltimore CyberTrust Root
Entrust Root Certification Authority
GeoTrust Global CA
GeoTrust Universal CA
GeoTrust Universal CA 2
AAA Certificate Services
QuoVadis Root Certification Authority
QuoVadis Root CA 2
QuoVadis Root CA 3
<Name(C=JP,O=SECOM Trust.net,OU=Security Communication RootCA1)>
Sonera Class2 CA
XRamp Global Certification Authority
<Name(C=US,O=The Go Daddy Group\, Inc.,OU=Go Daddy Class 2 Certification Authority)>
<Name(C=US,O=Starfield Technologies\, Inc.,OU=Starfield Class 2 Certification Authority)>
<Name(C=TW,O=Government Root Certification Authority)>
DigiCert Assured ID Root CA
DigiCert Global Root CA
DigiCert High Assurance EV Root CA
DST Root CA X3
SwissSign Gold CA - G2
SwissSign Silver CA - G2
GeoTrust Primary Certification Authority
thawte Primary Root CA
VeriSign Class 3 Public Primary Certification Authority - G5
SecureTrust CA
Secure Global CA
COMODO Certification Authority
Network Sol

These are trusted certificates that can be stored locally and used to verify certificates in the chain. Let's see how this is done.

### Establish a Socket Connection

We have seen how to establish a socket connection before.  The folks who implement TLS in Python have gone to a lot of trouble to make it transparent to use a TLS-enabled socket.  The problem is that the normal ways of doing so "hide" a lot of the details, and we explicitly want access to those details.  So we have to go to a bit of extra trouble to get this to work.

### Parameters to Use

In [40]:
# This is a server that has to get an updated certificate every year.
HOSTNAME = 'cybertruck1.engr.colostate.edu'
DEST_IPADDR = '129.82.20.206'
PORT = 443

To prepare for this section:
* Open Wireshark
* Set a capture filter to 'host 129.82.20.206 and port 443'
* Set a display filter to 'tls'
![WiresharkSetupforTLS.png](WiresharkSetupforTLS.png)

In [41]:
#Get the server's certificate based on a handshake routine.
server_pem = ssl.get_server_certificate((HOSTNAME,PORT),ssl.PROTOCOL_TLS)
server_pem

'-----BEGIN CERTIFICATE-----\nMIIGxjCCBa6gAwIBAgIQO+gndbZGxQAAibXK0Y+GqDANBgkqhkiG9w0BAQsFADB2\nMQswCQYDVQQGEwJVUzELMAkGA1UECBMCTUkxEjAQBgNVBAcTCUFubiBBcmJvcjES\nMBAGA1UEChMJSW50ZXJuZXQyMREwDwYDVQQLEwhJbkNvbW1vbjEfMB0GA1UEAxMW\nSW5Db21tb24gUlNBIFNlcnZlciBDQTAeFw0yMzAzMjAwMDAwMDBaFw0yNDA0MTky\nMzU5NTlaMG0xCzAJBgNVBAYTAlVTMREwDwYDVQQIEwhDb2xvcmFkbzEiMCAGA1UE\nChMZQ29sb3JhZG8gU3RhdGUgVW5pdmVyc2l0eTEnMCUGA1UEAxMeY3liZXJ0cnVj\nazEuZW5nci5jb2xvc3RhdGUuZWR1MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIB\nCgKCAQEA07ihTYI9/kRbUPujwkNa6e6p2YW5rTICJOIixnbes0PN/bbPOcQrrUF1\ntY5h5WFBTmMiSN4ZJBWShFii0a9LmNi5ciqwHtak2/8lfckoFMDNQpS3N2iKvyc4\nuMuNcyGlcn+z/OOYFza555uzeempvO7TNPggOfytnSw4PtQPwey6n3FdMx1Y4vB6\nGJJ+QZV/SYx5N/AbUFKr8sqEW3MxgvzcpVfH/AbQkVwLPBGBWDlci8jz1iloCN4d\nDWDlY/CeJ0vSnc0/CLB9arcXMo9Qcqlf4iPnqtkLb3zCFH63vo0GPnmHg6WcjHNK\nWH7xsuRFdk0Egj9Q5fwkM09D+wrBTwIDAQABo4IDVzCCA1MwHwYDVR0jBBgwFoAU\nHgWjd49sluJbh0umtIascQAM5zgwHQYDVR0OBBYEFGcMjNFBcvxs3GIsb3nqbbeo\n6Ju5MA4GA1UdDwEB/wQEAwIFoDAMBgNVHRMBAf8EAjAAMB

This produced a recording in wireshark.
![TLS1.2WiresharkCapture.png](TLS1.2WiresharkCapture.png)

In [78]:
#What does this certificate say?
server_certs = x509.load_pem_x509_certificates(server_pem.encode('ascii'))
for inter_cert in server_certs:
    print(inter_cert)

<Certificate(subject=<Name(C=US,ST=Colorado,O=Colorado State University,CN=cybertruck1.engr.colostate.edu)>, ...)>


There's only one certificate

In [79]:
context = ssl.create_default_context()
context.load_verify_locations(cafile=certifi.where())
context.check_hostname

True

In [80]:
context.options

<Options.OP_NO_SSLv3|OP_CIPHER_SERVER_PREFERENCE|OP_ENABLE_MIDDLEBOX_COMPAT|OP_NO_COMPRESSION|OP_ALL: -2108555180>

In [81]:
for c in context.get_ca_certs():
    for item in c['issuer']:
        if item[0][0] == 'commonName':
            print(item[0][1])

ISRG Root X1
Certigna
TeliaSonera Root CA v1
Izenpe.com
Sonera Class2 CA
Amazon Root CA 1
Amazon Root CA 2
Amazon Root CA 3
Amazon Root CA 4
Cybertrust Global Root
Atos TrustedRoot 2011
UCA Global G2 Root
DST Root CA X3
AffirmTrust Premium
GeoTrust Global CA
ACCVRAIZ1
AffirmTrust Commercial
AffirmTrust Networking
QuoVadis Root CA 2
QuoVadis Root CA 3
SwissSign Gold CA - G2
AffirmTrust Premium ECC
GeoTrust Universal CA
SwissSign Silver CA - G2
UCA Extended Validation Root
Hongkong Post Root CA 1
GeoTrust Universal CA 2
GTS Root R1
GTS Root R2
GTS Root R3
GTS Root R4
QuoVadis Root CA 1 G3
QuoVadis Root CA 2 G3
QuoVadis Root CA 3 G3
SecureTrust CA
IdenTrust Commercial Root CA 1
Secure Global CA
DigiCert Global Root CA
GlobalSign
GlobalSign
GlobalSign
D-TRUST Root Class 3 CA 2 2009
IdenTrust Public Sector Root CA 1
DigiCert Global Root CA
DigiCert Global Root CA
Buypass Class 2 Root CA
Buypass Class 3 Root CA
DigiCert Global Root CA
Baltimore CyberTrust Root
ISRG Root X1
D-TRUST Root Class

### Create the socket connection

The call to 'do_handshake' does all the work required establish the connection.

In [82]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    with context.wrap_socket(sock, server_hostname=HOSTNAME) as conn:
        conn.settimeout(5)
        conn.connect((DEST_IPADDR, PORT))
        conn.setblocking(1)
        conn.do_handshake()
        print(conn)
        cert = conn.getpeercert()
        cert_bin = conn.getpeercert(binary_form=True)

<ssl.SSLSocket fd=1480, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.125', 49609), raddr=('129.82.20.206', 443)>


In [83]:
cert

{'subject': ((('countryName', 'US'),),
  (('stateOrProvinceName', 'Colorado'),),
  (('organizationName', 'Colorado State University'),),
  (('commonName', 'cybertruck1.engr.colostate.edu'),)),
 'issuer': ((('countryName', 'US'),),
  (('stateOrProvinceName', 'MI'),),
  (('localityName', 'Ann Arbor'),),
  (('organizationName', 'Internet2'),),
  (('organizationalUnitName', 'InCommon'),),
  (('commonName', 'InCommon RSA Server CA'),)),
 'version': 3,
 'serialNumber': '3BE82775B646C5000089B5CAD18F86A8',
 'notBefore': 'Mar 20 00:00:00 2023 GMT',
 'notAfter': 'Apr 19 23:59:59 2024 GMT',
 'subjectAltName': (('DNS', 'cybertruck1.engr.colostate.edu'),),
 'OCSP': ('http://ocsp.usertrust.com',),
 'caIssuers': ('http://crt.usertrust.com/InCommonRSAServerCA_2.crt',),
 'crlDistributionPoints': ('http://crl.incommon-rsa.org/InCommonRSAServerCA.crl',)}

In [84]:
cert['caIssuers'][0]

'http://crt.usertrust.com/InCommonRSAServerCA_2.crt'

In [85]:
cert_bin[:300]

b'0\x82\x06\xc60\x82\x05\xae\xa0\x03\x02\x01\x02\x02\x10;\xe8\'u\xb6F\xc5\x00\x00\x89\xb5\xca\xd1\x8f\x86\xa80\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000v1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x0b0\t\x06\x03U\x04\x08\x13\x02MI1\x120\x10\x06\x03U\x04\x07\x13\tAnn Arbor1\x120\x10\x06\x03U\x04\n\x13\tInternet21\x110\x0f\x06\x03U\x04\x0b\x13\x08InCommon1\x1f0\x1d\x06\x03U\x04\x03\x13\x16InCommon RSA Server CA0\x1e\x17\r230320000000Z\x17\r240419235959Z0m1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x110\x0f\x06\x03U\x04\x08\x13\x08Colorado1"0 \x06\x03U\x04\n\x13\x19Colorado State University1\'0%\x06\x03U\x04\x03\x13\x1ecybertruck1.engr.colo'

### Alternatively, we can dig into the x509 object from cryptography.io

In [86]:
#Load the binary (DER) certificate
server_cert = x509.load_der_x509_certificate(cert_bin)
server_cert

<Certificate(subject=<Name(C=US,ST=Colorado,O=Colorado State University,CN=cybertruck1.engr.colostate.edu)>, ...)>

In [87]:
# Find out he common name for the issuer of the server's certificate.
for attribute in server_cert.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME):
    print(attribute.value)
issuer_name = attribute.value

InCommon RSA Server CA


In [88]:
#look through the extensions
for e in server_cert.extensions:
    print(e)

<Extension(oid=<ObjectIdentifier(oid=2.5.29.35, name=authorityKeyIdentifier)>, critical=False, value=<AuthorityKeyIdentifier(key_identifier=b'\x1e\x05\xa3w\x8fl\x96\xe2[\x87K\xa6\xb4\x86\xacq\x00\x0c\xe78', authority_cert_issuer=None, authority_cert_serial_number=None)>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.14, name=subjectKeyIdentifier)>, critical=False, value=<SubjectKeyIdentifier(digest=b'g\x0c\x8c\xd1Ar\xfcl\xdcb,oy\xeam\xb7\xa8\xe8\x9b\xb9')>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.15, name=keyUsage)>, critical=True, value=<KeyUsage(digital_signature=True, content_commitment=False, key_encipherment=True, data_encipherment=False, key_agreement=False, key_cert_sign=False, crl_sign=False, encipher_only=False, decipher_only=False)>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.19, name=basicConstraints)>, critical=True, value=<BasicConstraints(ca=False, path_length=None)>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.37, name=extendedKeyUsage)>, critical=False, value=<Ex

In [89]:
#Find the certificate authority issuers
issuers_constant = x509.oid.AuthorityInformationAccessOID.CA_ISSUERS
issuers_constant

<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>

In [90]:
access_constant = x509.ExtensionOID.AUTHORITY_INFORMATION_ACCESS
access_constant

<ObjectIdentifier(oid=1.3.6.1.5.5.7.1.1, name=authorityInfoAccess)>

In [91]:
server_cert.extensions.get_extension_for_oid(access_constant)

<Extension(oid=<ObjectIdentifier(oid=1.3.6.1.5.5.7.1.1, name=authorityInfoAccess)>, critical=False, value=<AuthorityInformationAccess([<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>, access_location=<UniformResourceIdentifier(value='http://crt.usertrust.com/InCommonRSAServerCA_2.crt')>)>, <AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.1, name=OCSP)>, access_location=<UniformResourceIdentifier(value='http://ocsp.usertrust.com')>)>])>)>

In [92]:
server_cert.extensions.get_extension_for_oid(access_constant).value

<AuthorityInformationAccess([<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>, access_location=<UniformResourceIdentifier(value='http://crt.usertrust.com/InCommonRSAServerCA_2.crt')>)>, <AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.1, name=OCSP)>, access_location=<UniformResourceIdentifier(value='http://ocsp.usertrust.com')>)>])>

In [93]:
for a in server_cert.extensions.get_extension_for_oid(access_constant).value:
    print(a)

<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>, access_location=<UniformResourceIdentifier(value='http://crt.usertrust.com/InCommonRSAServerCA_2.crt')>)>
<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.1, name=OCSP)>, access_location=<UniformResourceIdentifier(value='http://ocsp.usertrust.com')>)>


In [95]:
caIssuers = server_cert.extensions.get_extension_for_oid(access_constant).value[0].access_location.value
caIssuers

'http://crt.usertrust.com/InCommonRSAServerCA_2.crt'

## Download the intermediate certificate

In [96]:
#Use requests to get the bytes for the certificate. Often these are stored for faster lookups.
r = requests.get(caIssuers)
r.content

b'0\x82\x05\xf90\x82\x03\xe1\xa0\x03\x02\x01\x02\x02\x10G \xd0\xfa\x85F\x1a~\x17\xa1d\x02\x91\x84ct0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0c\x05\x000\x81\x881\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x130\x11\x06\x03U\x04\x08\x13\nNew Jersey1\x140\x12\x06\x03U\x04\x07\x13\x0bJersey City1\x1e0\x1c\x06\x03U\x04\n\x13\x15The USERTRUST Network1.0,\x06\x03U\x04\x03\x13%USERTrust RSA Certification Authority0\x1e\x17\r141006000000Z\x17\r241005235959Z0v1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x0b0\t\x06\x03U\x04\x08\x13\x02MI1\x120\x10\x06\x03U\x04\x07\x13\tAnn Arbor1\x120\x10\x06\x03U\x04\n\x13\tInternet21\x110\x0f\x06\x03U\x04\x0b\x13\x08InCommon1\x1f0\x1d\x06\x03U\x04\x03\x13\x16InCommon RSA Server CA0\x82\x01"0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x82\x01\x0f\x000\x82\x01\n\x02\x82\x01\x01\x00\x9c\x1b\xf1\xbb/\x7fc\x18\x15QQT\x0f\x9e\xc5NM\x10X\xfa0\x9b\x17)\x90\xe63\x0c\xac\x13S|T\x91\xb4\xea\xd8n\x9b\x89m\xbb3>\x8f\xd2\r\xa6\xe9\xf9\xba\xe9\r\x0c\x1a\x9e\xb2\x8e\xc9p.\xef\x1e\x05}\x95\xeb-\

In [101]:
intermediate_cert = x509.load_der_x509_certificate(r.content)
intermediate_cert

<Certificate(subject=<Name(C=US,ST=MI,L=Ann Arbor,O=Internet2,OU=InCommon,CN=InCommon RSA Server CA)>, ...)>

In [102]:
#generate a list of known roots
root_names = []
for c in root_certs:
    try:
        root_names.append(c.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME)[-1].value)
    except IndexError:
        pass
root_names

['GlobalSign Root CA',
 'GlobalSign',
 'Entrust.net Certification Authority (2048)',
 'Baltimore CyberTrust Root',
 'Entrust Root Certification Authority',
 'GeoTrust Global CA',
 'GeoTrust Universal CA',
 'GeoTrust Universal CA 2',
 'AAA Certificate Services',
 'QuoVadis Root Certification Authority',
 'QuoVadis Root CA 2',
 'QuoVadis Root CA 3',
 'Sonera Class2 CA',
 'XRamp Global Certification Authority',
 'DigiCert Assured ID Root CA',
 'DigiCert Global Root CA',
 'DigiCert High Assurance EV Root CA',
 'DST Root CA X3',
 'SwissSign Gold CA - G2',
 'SwissSign Silver CA - G2',
 'GeoTrust Primary Certification Authority',
 'thawte Primary Root CA',
 'VeriSign Class 3 Public Primary Certification Authority - G5',
 'SecureTrust CA',
 'Secure Global CA',
 'COMODO Certification Authority',
 'Network Solutions Certificate Authority',
 'COMODO ECC Certification Authority',
 'OISTE WISeKey Global Root GA CA',
 'Certigna',
 'Cybertrust Global Root',
 'GeoTrust Primary Certification Authority 

In [103]:
issuer_name in root_names

False

In [104]:
intermediate_cert.subject

<Name(C=US,ST=MI,L=Ann Arbor,O=Internet2,OU=InCommon,CN=InCommon RSA Server CA)>

In [105]:
intermediate_cert.issuer

<Name(C=US,ST=New Jersey,L=Jersey City,O=The USERTRUST Network,CN=USERTrust RSA Certification Authority)>

In [106]:
for attribute in intermediate_cert.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME):
    print(attribute.value)
issuer_name = attribute.value

USERTrust RSA Certification Authority


In [107]:
#Check to see if the issuer of the intermetidate certificate is in a root of trust.
issuer_name in root_names

True

Finally, we have reached a root certificate. 

In [115]:
#Load the correct certificate from the locally stored list of root CA certificates.

for c in root_certs:
    try:
        root_name = c.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME)[-1].value
        if root_name == issuer_name:
            break
    except IndexError:
        pass
rootCA = c
rootCA

<Certificate(subject=<Name(C=US,ST=New Jersey,L=Jersey City,O=The USERTRUST Network,CN=USERTrust RSA Certification Authority)>, ...)>

In [120]:
#extract the details from the intermediate certificate
intermediate_cert

<Certificate(subject=<Name(C=US,ST=MI,L=Ann Arbor,O=Internet2,OU=InCommon,CN=InCommon RSA Server CA)>, ...)>

#### Verify the certificate with the public key from our local cert store

In [121]:
#
# Verify intermediate certificate w/ rootCA public key (from the trust store)
#
try:
    rootCA.public_key().verify(
            intermediate_cert.signature,
            intermediate_cert.tbs_certificate_bytes,
            padding.PKCS1v15(),
            intermediate_cert.signature_hash_algorithm
        )
    print("Valid signature")
except InvalidSignature as e:
    print("Invalid signature")

Valid signature


In [122]:
# Recall
# Verify intermediate certificate w/ rootCA public key (from the trust store)
#
try:
    intermediate_cert.public_key().verify(
            server_cert.signature,
            server_cert.tbs_certificate_bytes,
            padding.PKCS1v15(),
            server_cert.signature_hash_algorithm
        )
    print("Valid signature")
except InvalidSignature as e:
    print("Invalid signature")

Valid signature


#### Verify the hostname

In [134]:
# Recall the hostname
server_cert.subject

<Name(C=US,ST=Colorado,O=Colorado State University,CN=cybertruck1.engr.colostate.edu)>

In [135]:
ext = server_cert.extensions.get_extension_for_oid(x509.ExtensionOID.SUBJECT_ALTERNATIVE_NAME)
ext.value.get_values_for_type(x509.DNSName)

['cybertruck1.engr.colostate.edu']

In [136]:
#Check to see if the client hostname is in the certificate
HOSTNAME in ext.value.get_values_for_type(x509.DNSName)

True

#### Verify the time

In [137]:
# Check for a valid time slot:
if (datetime.utcnow() < server_cert.not_valid_after and
    datetime.utcnow() > server_cert.not_valid_before):
    print("The certificate has not expired")
else:
    print("The certificate is outside the allowed times.")

The certificate has not expired


# Summary
In this notebook, we went through some ideas of transport layer security (TLS) and went through the process of issuing and verifying x509 Certificates. 